In [1]:
import numpy as np
import torch 
import torch.nn as nn 
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

# LeNet

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=5)
        self.conv2 = nn.Conv2d(8, 20, kernel_size=5)
        self.pooling = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)
    def forward(self, x):
        batch_size = x.size(0)
        x = self.conv1(x)
#         print( list(  x.size()  ) )
        x = self.pooling(x)
#         print( list(  x.size()  ) )
        x = F.relu(x)
        
        x = self.conv2(x)
#         print( list(  x.size()  ) )
        x = self.pooling(x)
#         print( list(  x.size()  ) )
        x = F.relu(x)
        x = x.view(batch_size, -1)
        x = self.fc(x)
        return x

# model = Net()
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.to(device)

(W - F + 2P)/S + 1

[不同的损失函数对比](https://blog.csdn.net/f156207495/article/details/88658009)

## GoogleLeNet

In [3]:
class Inception(nn.Module):
    def __init__(self, in_channels):
        super(Inception, self).__init__()
        
        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)
        
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        
        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)
        
        self.branch3x3_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)
        
        
        
    def forward(self, x):
        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)
        
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)
        outputs = [branch1x1, branch5x5, branch3x3, branch_pool]
        return torch.cat(outputs, dim=1)


class GoogleLeNet(nn.Module):
    def __init__(self):
        super(GoogleLeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)
        
        self.incep1 = Inception(in_channels=10)
        self.incep2 = Inception(in_channels=20)
        
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incep1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incep2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

# model = GoogleLeNet()

# Residual Network

In [7]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.channels = in_channels
        self.conv1 = nn.Conv2d(in_channels, in_channels,
                              kernel_size=3,
                              padding=1)
        self.conv2 = nn.Conv2d(in_channels, in_channels,
                              kernel_size=3,
                              padding=1)
    def forward(self, x):
        y = F.relu(self.conv1(x))
        y = self.conv2(y)
        return F.relu(x+y)

class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.rblock1 = ResidualBlock(16)
        self.rblock2 = ResidualBlock(32)
        self.fc = nn.Linear(512, 10)
    def forward(self, x):
        in_size = x.size(0)
        x = self.mp(F.relu(self.conv1(x)))
        x = self.rblock1(x)
        x = self.mp(F.relu(self.conv2(x)))
        x = self.rblock2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x


# DenseNet

In [11]:
model = DenseNet(10)

batch_size = 8
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307, ), (0.3081,))
])
train_datasets = datasets.MNIST(
    root="../../dataset/mnist",
    train=True,
    download=True,
    transform=transform
)
train_loader = DataLoader(train_datasets, shuffle=True, batch_size=batch_size)
test_datasets = datasets.MNIST(root="../../dataset/mnist",
                               train=False,
                               download=True,
                               transform=transform)
test_loader = DataLoader(test_datasets, shuffle=True, batch_size=batch_size)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
#         inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 500 == 0:
            print('[%d, %5d] loss : %.3f' % (epoch + 1, batch_idx + 1, running_loss/300))
            running_loss = 0.
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data 
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print("Accuracy on test set : %d %%" % ( 100 *correct / total))
# if __name__ == '__main__':
# model = Net()
for epoch in range(1):
    train(epoch)
    test()


RuntimeError: Given groups=1, weight of size 64 3 7 7, expected input[8, 1, 28, 28] to have 3 channels, but got 1 channels instead

In [7]:
len(train_datasets)

60000

In [ ]:
train_datasets[0][0].shape

In [ ]:
len(test_datasets)